# Symmetrical choiceset:

We want to symmetrically sample both sides of the subjective equality point. 
That means having equal amount of trials in each side of the SV difference. 

This version of the code doesn't use python external libraries. The reason is to optimize the code to make it compatible with online based tasks that use Js. 

JS script data types we are going to use:
- Arrays: written with square brackets. Items are separated by commas. This is analogous to python lists. 

    ``` const cars = ["Saab", "Volvo", "BMW"]; ```
- Objects: JavaScript objects are written with curly braces {}.

Object properties are written as name:value pairs, separated by commas.

This is analogous to python dictionaries. I intend to use them to create the desired choicet. 

        const person = {firstName:"John", lastName:"Doe",  age:50, eyeColor:"blue"}

In [70]:
import csv #This is the only library I will use. However, it is built in python. 
post_mean_gain = [0.45456, 1.0343]
post_mean_loss = [0.896596, -0.765]


The idea is to create an output CSV without using pandas :(
    We are going to do this by using lists and dicitonaries. The idea is to creaate the trials as lists of dictionries

With that, we can create the CSV at the end
```
Trials = 
{'category': ['Zero_Reward',  'Zero_Reward',..., 'Int_neg', 'Int_neg','Int_neg'],
 'p_reward': [0.75, 0.5,0.38, 0.25, 0.13, ..., 0.75 0.5, 0.38,0.25,0.13],
 'value_reward': [-0.0,  -5.71, ..., -22.453 , -34.884],
 'ambiguity': [0.0, 0.0,0.0  0.24, 0.5, 0.74, 0.0, 0.0],
 'SV_reward': [0.0, -5.551115123125783e-17, -1.813, -3.62, -2.222, -4.532],
 'SV_New_Safe': [-2.719, -1.8130],
 'value_lott_SE': [-12.723, -12.723, -12.72,
  'value_safe': [-7.20, -0.226,  -2.995, -2.7-2.51]
 }
```

# 1. Input: ADO parameters

In [71]:
alpha_pos, beta_pos = post_mean_gain[0], post_mean_gain[1] #parameters for gains
alpha_neg, beta_neg = post_mean_loss[0], post_mean_loss[1] #parameters for losses

In [72]:
#Declare the variables
p_options = [0.75, 0.5, 0.38, 0.25, 0.13]
amb_options = [0.24, 0.5, 0.74]
A_null = 0.0 # no ambiguity
Vmax = 50.0 #The maximum amount we can pay to participants
Vmin = -50.0 #The minimum amount we can take from participants
Vsafe_pos = 5.0 # The old safe option in positive trials
Vsafe_neg = -5.0 # The old safe option in negative trials

#First, we create an empty dictionary that we will populate with the trials info. 

empty_trials = { #An indivual trial consists of the values with the same index in the different lists
  'category': [],
  'p_reward': [],
  'ambiguity': [],
  'value_reward': [],
  'SV_reward': [],
  'value_safe': [],
  'SV_safe': []
}
ordered_keys = empty_trials.keys()

In [83]:
##FUNCTIONS

#Create the extreme positive gains and losses
def get_extr_rows(p_options, domain='gain'):
    """
    Inputs:
        - p_options = list of probability values (float)
    Output:
        - dictionaries of lists containing the maximum gain or loss. With empty keys that we will populate later on with SV and Vsafw
    """
    extr = empty_trials.copy()
    category = 'Extr_pos' if domain == 'gain' else 'Extr_neg'
    V=[Vmax] * len(p_options) #A list of values with the same length as our p_options
    if domain == 'loss':
            V = [Vmin] * len(p_options)
    extr = {
            'category': [category]*len(p_options),
            'p_reward': p_options,
            'ambiguity': [0.0]*len(p_options),
            'value_reward': V,
        }
    return extr

#Add ambiguity
def add_ambig(trials,prob_amb=0.50):
    """
    Inputs:
        - trials dictionary
        - amb_options = list of possible ambiguity values
    Output:
        - initial trials dictionary with new ambiguity trials
    """
    V = [trials['value_reward'][0]]*len(amb_options)
    amb_trials = {
            'p_reward': [prob_amb]*len(amb_options),
            'value_reward': V,
            'ambiguity': [a for a in amb_options],
        }
    final_dict = {k: trials[k] + amb_trials[k] for k in set(trials) & set(amb_trials)}
    final_dict['category'] = [trials['category'][0]]*len(final_dict['value_reward'][:])
    return final_dict

#Add Subjective value given participant's alpha and beta
def add_SV(trials, alpha, beta, domain='gain'):
    """
    Inputs:
        - trials dictionary
        - Model parameters
    Output:
        - trials dictionary with subjective value 
    """
    Amp = 1.0
    if domain == 'loss':
          Amp = -1.0
    trials['SV_reward'] = [(p -(beta*A/2)) * (Amp*abs(v) ** alpha) for p, v, A 
                         in zip(trials['p_reward'], trials['value_reward'], trials['ambiguity'])] #This ZIP method is going to be a PIA in JS. However, I think we can emulate it in JS from scratch 
    return trials  

# Add Vsafe and its subjective value     
def add_Vsafe(trials, alpha,beta, domain = 'gain'):
    """
     This function creates appends the new values of safe options to the Extreme reward or loss dictionary. The subjective value of the safe is half of that 
     of the maximum gain or loss SVnew_safe = 0.5*SV_max for each probability.
     Inputs:
        - trials dictionary
        - Model parameters
    Output:
        - Appeneded New safe options to extreme trials
        - SV of SE trials
    """ 
    Amp = 1.0 
    if domain == 'loss':
        Amp = -1.0
    trials['SV_safe'] = [sv/2 for sv in trials['SV_reward']]
    trials['value_safe'] = [Amp*(abs(sv))**(1/alpha) for sv in trials['SV_safe']]
    return trials

def get_safe_trials (extr, alpha, beta, domain = 'gain'):
    """
    This function creates new values of safe options whose subjective value is half of that 
    of the maximum gain or loss SVnew_safe = 0.5*SV_max for each probability. That SV is going to be the SV of subjective equality 

    Inputs:
        - Extreme trials dictionary
        - Model parameters
    Output:
        - New safe options
        - SV of SE trials
    """
    safe_options = {}
    Amp = 1.0 
    if domain == 'loss':
        Amp = -1.0
    safe_options['p_reward'] = [p for p in extr['p_reward']]
    safe_options['ambiguity'] = [a for a in extr['ambiguity']]
    safe_options['SV_reward'] = [sv/2 for sv in extr['SV_reward']]
    safe_options['value_safe'] = [Amp*(abs(sv))**(1/alpha) for sv in safe_options['SV_reward']]
    return safe_options 
     
def get_SE_trials (trials, alpha, beta, domain = 'gains'):
    '''     
     Creates trials around the point of subjective equality

     Inputs:
     -Max reward or loss trials
     -Model Parameters 
    '''
    SE_trials = {}
    SE_trials['category'] = ['SE_pos']*len(trials['SV_safe'])
    Amp = 1.0 
    if domain == 'loss':
        Amp = -1.0
        SE_trials['category'] = ['SE_neg']*len(trials['SV_safe'])    
    SE_trials['p_reward'] = [p for p in trials['p_reward']]
    SE_trials['ambiguity'] = [a for a in trials['ambiguity']]
    SE_trials['SV_reward'] = [sv/2 for sv in trials['SV_reward']]
    SE_trials['value_safe'] = [Amp*(abs(sv))**(1/alpha) for sv in trials['SV_safe']]
    SE_trials['value_reward'] = [(Amp*(abs(sv)/((p) - (beta*A/2)))**(1/alpha)) for sv, p, A
                                  in zip(trials['SV_safe'], trials['p_reward'], trials['ambiguity'])]
    return SE_trials

def get_cent_SE_trials(SE_trials, domain = 'gain'):
     cent_SE= {}
     
     SE = {key: SE_trials[key] for key in ['p_reward', 'ambiguity', 'value_safe', 'value_reward']}
     cent_SE_trial_plus2 = {key: SE_trials[key] for key in ['p_reward', 'ambiguity', 'value_safe', 'value_reward']}
     cent_SE_trial_plus2['value_reward'] = [value_reward + 2.0 for value_reward in cent_SE_trial_plus2['value_reward']]

     cent_SE_trial_minus2 = {key: SE_trials[key] for key in ['p_reward', 'ambiguity', 'value_safe', 'value_reward']}
     cent_SE_trial_minus2['value_reward'] = [value_reward - 2.0 for value_reward in cent_SE_trial_minus2['value_reward']]

     # Merge dictionaries while keeping values in lists
     cent_SE = SE.copy()  # Start with the SE dictionary

    # Update values in cent_SE with values from cent_SE_trial_plus2
     for key, value in cent_SE_trial_plus2.items():
        cent_SE[key] = cent_SE[key] + value

     for key, value in cent_SE_trial_minus2.items():
        cent_SE[key] = cent_SE[key] + value

     cent_SE['category'] = ['cent_SE_pos']*len(cent_SE['value_reward']) 
     if domain == 'loss':
        cent_SE['category'] = ['cent_SE_neg']*len(cent_SE['value_reward'])  
     return cent_SE

def flatten(l):
    return [item for sublist in l for item in sublist]

def get_interm_trials(rows_extr, alpha, beta,num_steps=2, domain='gain'):
    '''     
     Creates intermediate trials between subjective equality and extreme

     Inputs:
     -Max reward or loss trials
     -Model Parameters 
    '''    
    full_int = {}

    SV_step =  [(reward-safe)/(num_steps + 1) for (reward,safe) in zip(rows_extr['SV_reward'],rows_extr['SV_safe'])]
    list_x = [i for i in range(-num_steps-1,num_steps+1) if i != 0]
    list_dict = []
    for x in list_x:
        interm = {}
        Amp,category = 1.0,'IntPos'
        if domain == 'loss':
            Amp,category = -1.0,'IntNeg'
        if x == -num_steps-1:
            category = 'Zero_Reward'
        interm['category'] = [category]*len(rows_extr['category'])
        interm['p_reward'] = rows_extr['p_reward']
        interm['ambiguity'] = rows_extr['ambiguity']
        interm['SV_reward'] = [safe+x*step for (safe,step) in zip(rows_extr['SV_safe'],SV_step)]
        interm['value_safe'] = rows_extr['value_safe']
        interm['SV_safe'] = rows_extr['SV_safe']
        interm['value_reward'] = [(Amp*(abs(sv)/((p) - (beta*A/2)))**(1/alpha)) for sv, p, A
                                      in zip(interm['SV_safe'], interm['p_reward'], interm['ambiguity'])]
        list_dict += [interm]
    full_int = {k :flatten([d[k] for d in list_dict]) for k in list_dict[0]}
    return full_int

In [74]:
def order_keys(dict,keys):
    return {k: dict[k] for k in keys if k in dict.keys()}

In [75]:
##GAINS
rows_extr_pos = get_extr_rows(p_options)
rows_extr_pos = order_keys(rows_extr_pos,ordered_keys)
print(rows_extr_pos)

rows_extr_pos = add_ambig(rows_extr_pos)
rows_extr_pos = order_keys(rows_extr_pos,ordered_keys)
print(rows_extr_pos)

rows_extr_pos = add_SV(rows_extr_pos, alpha_pos,beta_pos) 
rows_extr_pos = order_keys(rows_extr_pos,ordered_keys)
print(rows_extr_pos)

rows_extr_pos = add_Vsafe(rows_extr_pos, alpha_pos,beta_pos)
rows_extr_pos = order_keys(rows_extr_pos,ordered_keys)
print(rows_extr_pos)


{'category': ['Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos'], 'p_reward': [0.75, 0.5, 0.38, 0.25, 0.13], 'ambiguity': [0.0, 0.0, 0.0, 0.0, 0.0], 'value_reward': [50.0, 50.0, 50.0, 50.0, 50.0]}
{'category': ['Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos'], 'p_reward': [0.75, 0.5, 0.38, 0.25, 0.13, 0.5, 0.5, 0.5], 'ambiguity': [0.0, 0.0, 0.0, 0.0, 0.0, 0.24, 0.5, 0.74], 'value_reward': [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]}
{'category': ['Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos', 'Extr_pos'], 'p_reward': [0.75, 0.5, 0.38, 0.25, 0.13, 0.5, 0.5, 0.5], 'ambiguity': [0.0, 0.0, 0.0, 0.0, 0.0, 0.24, 0.5, 0.74], 'value_reward': [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0], 'SV_reward': [4.439612531232422, 2.9597416874882816, 2.249403682491094, 1.4798708437441408, 0.7695328387469532, 2.2250390889196905, 1.4291112738037168, 0.6944086752351256]}
{'category': ['Extr_pos', 'Extr_pos',

In [79]:

#Safe options dictionary.
safe_pos = get_safe_trials(rows_extr_pos, alpha_pos, beta_pos)
safe_pos = order_keys(safe_pos,ordered_keys)
print(safe_pos)

#Subjective equality trials
SE_pos = get_SE_trials(rows_extr_pos,alpha_pos,beta_pos)
SE_pos = order_keys(SE_pos,ordered_keys)
print(SE_pos)

# cent_pos contains SE_pos within it... SE_pos not needed later on
cent_pos = get_cent_SE_trials(SE_pos)
cent_pos = order_keys(cent_pos,ordered_keys)
print(cent_pos)

{'p_reward': [0.75, 0.5, 0.38, 0.25, 0.13, 0.5, 0.5, 0.5], 'ambiguity': [0.0, 0.0, 0.0, 0.0, 0.0, 0.24, 0.5, 0.74], 'SV_reward': [2.219806265616211, 1.4798708437441408, 1.124701841245547, 0.7399354218720704, 0.3847664193734766, 1.1125195444598452, 0.7145556369018584, 0.3472043376175628], 'value_safe': [5.779214056024037, 2.3685382742811, 1.2950266345451085, 0.5155082363421659, 0.12231027909596252, 1.2643682118988953, 0.47740788166001397, 0.09757075252551307]}
{'category': ['SE_pos', 'SE_pos', 'SE_pos', 'SE_pos', 'SE_pos', 'SE_pos', 'SE_pos', 'SE_pos'], 'p_reward': [0.75, 0.5, 0.38, 0.25, 0.13, 0.5, 0.5, 0.5], 'ambiguity': [0.0, 0.0, 0.0, 0.0, 0.0, 0.24, 0.5, 0.74], 'value_reward': [10.882413046473419, 10.882413046473422, 10.882413046473422, 10.882413046473422, 10.882413046473422, 10.882413046473422, 10.882413046473422, 10.882413046473422], 'SV_reward': [2.219806265616211, 1.4798708437441408, 1.124701841245547, 0.7399354218720704, 0.3847664193734766, 1.1125195444598452, 0.71455563690185

In [84]:
int_pos = get_interm_trials(rows_extr_pos, alpha_pos, beta_pos)
int_pos = order_keys(int_pos,ordered_keys)
print(int_pos)

{'category': ['Zero_Reward', 'Zero_Reward', 'Zero_Reward', 'Zero_Reward', 'Zero_Reward', 'Zero_Reward', 'Zero_Reward', 'Zero_Reward', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos', 'IntPos'], 'p_reward': [0.75, 0.5, 0.38, 0.25, 0.13, 0.5, 0.5, 0.5, 0.75, 0.5, 0.38, 0.25, 0.13, 0.5, 0.5, 0.5, 0.75, 0.5, 0.38, 0.25, 0.13, 0.5, 0.5, 0.5, 0.75, 0.5, 0.38, 0.25, 0.13, 0.5, 0.5, 0.5, 0.75, 0.5, 0.38, 0.25, 0.13, 0.5, 0.5, 0.5], 'ambiguity': [0.0, 0.0, 0.0, 0.0, 0.0, 0.24, 0.5, 0.74, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24, 0.5, 0.74, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24, 0.5, 0.74, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24, 0.5, 0.74, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24, 0.5, 0.74], 'value_reward': [10.882413046473419, 10.882413046473422, 10.882413046473422, 10.882413

In [ ]:
##LOSSES
rows_extr_neg = get_extr_rows(p_options, domain = 'loss')
rows_extr_neg = add_ambig(rows_extr_neg) 
rows_extr_neg = add_SV(rows_extr_neg, alpha_neg,beta_neg, domain = 'loss')
rows_extr_neg = add_Vsafe(rows_extr_neg, alpha_neg,beta_neg)
#Neg Safe options
safe_neg = get_safe_trials(rows_extr_neg, alpha_neg, beta_neg, domain= 'loss')

#Subjective equality trials
SE_neg = get_SE_trials(rows_extr_neg, alpha_neg, beta_neg, domain='loss')
cent_neg = get_cent_SE_trials(SE_neg, domain = 'loss')
cent_neg

# Output CSV

In [ ]:
# Extract column names from the keys of the first dictionary
#fieldnames = crdm_trials[0].keys()

# Define the name of the CSV file
#csv_filename = 'crdm_trials.csv'

# Write the data to the CSV file
#with open(csv_filename, mode='w', newline='') as csv_file:
 #   writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    # Write the header row
  #  writer.writeheader()
    
    # Write the data rows
   # for row in crdm_trials:
    #    writer.writerow(row)

# print(f"CSV file '{csv_filename}' has been created.")